## Wielomiany interpolujące

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sym
from sympy import latex
from sympy.printing import cxxcode

try:
    from sympy.printing.cxx import CXX11CodePrinter
except ImportError:
    from sympy.printing.cxxcode import CXX11CodePrinter

import re

from IPython.display import display, Latex

# Równanie dyfuzji nośników

Równanie do rozwiązania ma postać:
$$
D \Delta u - A u - B u^2 - C u^3 + J = 0,
$$
gdzie
$$
J = \frac{j_\perp}{q_e\,d},
$$
w którym $j_\perp$ to wstrzykiwana gęstość prądu a $d$ to grubość złącza. Po zamianie na równanie linowe (korzystając z rozwinięcia w szereg Taylora) mamy:
$$
D \Delta u - E u + F = 0,
$$
gdzie
\begin{align}
E & = A + 2 B u_0 + 3 C u_0^2, \\
F & = B u_0^2 + 2 C u_0^3 + J_0.
\end{align}

In [2]:
A, B, C, D, = sym.symbols('A B C D')

Zastosujmy metodę Ritza w we współrzędnych kartezjańskich

In [3]:
x, y = sym.symbols('x y')
X, Y = sym.symbols('X Y')

ξ = x / X
ζ = y / Y

Jako funkcje próbne Używamy wielomianów trzeciego rzędu, które pozwalają na zagwarantowanie ciągłości funkcji i pochodnych:

In [4]:
Φ = np.linalg.inv(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 1, 1, 1],
    [0, 1, 2, 3]
])).T

φx = [sum(int(c) * ξ**i for i,c in enumerate(p)) for p in Φ]
φx[1] *= X
φx[3] *= X
φx = np.array([sym.expand(a) for a in φx])

φy = [sum(int(c) * ζ**i for i,c in enumerate(p)) for p in Φ]
φy[1] *= Y
φy[3] *= Y
φy = np.array([sym.expand(a) for a in φy])

## Funkcja przybliżana

Zakładamy, że
$$
u(x,y) = \sum_{i,j} u^{i,j} \, \varphi_{i,j}(x,y) \qquad u_0(x,y) = \sum_{i,j} U^{i,j} \,\varphi_{i,j}(x,y)
$$
gdzie
\begin{align}
u^{0,0}, u^{1,0}, u^{0,1} & = u(0,0), \left.\tfrac{du}{dx}\right|_{0,0}, \left.\tfrac{du}{dy}\right|_{0,0} \\
u^{2,0}, u^{3,0}, u^{2,1} & = u(X,0), \left.\tfrac{du}{dx}\right|_{X,0}, \left.\tfrac{du}{dy}\right|_{X,0} \\
u^{0,2}, u^{1,2}, u^{0,3} & = u(0,Y), \left.\tfrac{du}{dx}\right|_{0,Y}, \left.\tfrac{du}{dy}\right|_{0,Y} \\
u^{2,2}, u^{3,2}, u^{2,3} & = u(X,Y), \left.\tfrac{du}{dx}\right|_{X,Y}, \left.\tfrac{du}{dy}\right|_{X,Y} \\
\end{align}


Interpolacja musi być zrobiona tak, była sumetryczna względem zamiany $x$ i $y$. W związku z tym musi ona mieć postać
\begin{alignat}{9}
u(x,y) & = \varphi_0(x)\,\varphi_0(y)\,u^{0,0} && + \varphi_1(x)\,\varphi_0(y)\,u^{1,0} && + \varphi_2(x)\,\varphi_0(y)\,u^{2,0} && + \varphi_3(x)\,\varphi_0(y)\,u^{3,0} \\
       & + \varphi_0(x)\,\varphi_1(y)\,u^{0,1} &&                                       && + \varphi_2(x)\,\varphi_1(y)\,u^{2,1} && \\
       & + \varphi_0(x)\,\varphi_2(y)\,u^{0,2} && + \varphi_1(x)\,\varphi_2(y)\,u^{1,2} && + \varphi_2(x)\,\varphi_2(y)\,u^{2,2} && + \varphi_3(x)\,\varphi_2(y)\,u^{3,2} \\
       & + \varphi_0(x)\,\varphi_3(y)\,u^{0,3} &&                                       && + \varphi_2(x)\,\varphi_3(y)\,u^{2,3}. &&
\end{alignat}

In [5]:
φ = np.array([
    [φx[0] * φy[0], φx[1] * φy[0], φx[2] * φy[0], φx[3] * φy[0]],
    [φx[0] * φy[1],             0, φx[2] * φy[1],             0],
    [φx[0] * φy[2], φx[1] * φy[2], φx[2] * φy[2], φx[3] * φy[2]],
    [φx[0] * φy[3],             0, φx[2] * φy[3],             0]
])

idx = [(0,0), (0,1), (0,2), (0,3), (1,0), (1,2), (2,0), (2,1), (2,2), (2,3), (3,0), (3,2)]
φ = np.array([φ[idx[i]] for i in range(12)])

In [6]:
dφx = np.array([sym.expand(sym.diff(a, x)) for a in φ])
dφy = np.array([sym.expand(sym.diff(a, x)) for a in φ])

In [7]:
U = sym.IndexedBase('U', shape=(4,4))
u0 = sum(U[idx[k]] * φ[k] for k in range(12))
u0

(3*x**2/X**2 - 2*x**3/X**3)*(3*y**2/Y**2 - 2*y**3/Y**3)*U[2, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(-y**2/Y + y**3/Y**2)*U[3, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 2] + (3*x**2/X**2 - 2*x**3/X**3)*(y - 2*y**2/Y + y**3/Y**2)*U[1, 2] + (-x**2/X + x**3/X**2)*(3*y**2/Y**2 - 2*y**3/Y**3)*U[2, 3] + (-x**2/X + x**3/X**2)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 3] + (3*y**2/Y**2 - 2*y**3/Y**3)*(1 - 3*x**2/X**2 + 2*x**3/X**3)*U[2, 0] + (3*y**2/Y**2 - 2*y**3/Y**3)*(x - 2*x**2/X + x**3/X**2)*U[2, 1] + (-y**2/Y + y**3/Y**2)*(1 - 3*x**2/X**2 + 2*x**3/X**3)*U[3, 0] + (1 - 3*x**2/X**2 + 2*x**3/X**3)*(1 - 3*y**2/Y**2 + 2*y**3/Y**3)*U[0, 0] + (1 - 3*x**2/X**2 + 2*x**3/X**3)*(y - 2*y**2/Y + y**3/Y**2)*U[1, 0] + (1 - 3*y**2/Y**2 + 2*y**3/Y**3)*(x - 2*x**2/X + x**3/X**2)*U[0, 1]

## Interpolacja prądu

Aby dobrze odzdwierciedlić rozpływ prądu, zakładamy, że na elemencie zmienia się on co najwyżej liniowo

In [8]:
# z = np.array([L / 4, L * 3 / 4])
zx = np.array([0, X])
zy = np.array([0, Y])

Teraz zakładmy, że w tych punktach mamy wartości funkcji $J$. Budujemy wielomiany interpolacyjne.

In [9]:
J = sym.IndexedBase('J', shape=(2,2))

def La(i, x, z):
    res = 1
    for j in range(len(z)):
        if j == i: continue
        res *= (x - z[j]) / (z[i] - z[j])
    return res

In [10]:
Jxy = sym.simplify(sum(J[i,j] * La(i, x, zx) * La(j, y, zy) for i in range(len(zx)) for j in range(len(zy))))
Jxy

(x*y*J[1, 1] + x*(Y - y)*J[1, 0] + y*(X - x)*J[0, 1] + (X - x)*(Y - y)*J[0, 0])/(X*Y)

# Macierze sztywności i wektor obciążeń

Macierz sztywności budujemy z równania
$$
\left( - D \Delta + A + 2 B u_0 + 3 C u_0^2 \right) u = B u_0^2 + 2 C u_0^3 + J_0.
$$
Pamiętając, że
$$
u(x,y) = \sum_{i_x,i_y} u^{i_x,i_y} \, \varphi_{i_x,i_y}(x,y)
$$
i stosując metodę Ritza mamy
$$
\sum_{j_x,j_y} \int_0^L \left[ D \left( \frac{d\varphi_{i_x,i_y}}{dx} \frac{d\varphi_{j_x,j_y}}{dx} + \frac{d\varphi_{i_x,i_y}}{dy} \frac{d\varphi_{j_x,j_y}}{dy} \right) + \left( A + 2 B u_0 + 3 C u_0^2 \right) \varphi_{i_x,i_y} \varphi_{j_x,j_y} \right] u^j = \left( B u_0^2 + 2 C u_0^3 + J_0 \right) \varphi_{i_x,i_y}.
$$

Do tego, stosujemy
$$
u_0(x,y) = \sum_{i_x,i_y} U^{i_x,i_y} \,\varphi_{i_x,i_y}(x,y).
$$


In [11]:
execfile('matrices2.py')

In [12]:
K = KD + KA + KB + KC
K

Matrix([
[169*A*X*Y/1225 + B*X*Y*(4171*X*U[0, 1] - 1849*X*U[0, 3] + 873*X*U[2, 1] - 387*X*U[2, 3] + 4171*Y*U[1, 0] + 873*Y*U[1, 2] - 1849*Y*U[3, 0] - 387*Y*U[3, 2] + 33282*U[0, 0] + 6966*U[0, 2] + 6966*U[2, 0] + 1458*U[2, 2])/176400 + C*X*Y*(80861760*X**2*U[0, 1]**2 - 74797128*X**2*U[0, 1]*U[0, 3] + 24222240*X**2*U[0, 1]*U[2, 1] - 11202786*X**2*U[0, 1]*U[2, 3] + 25798752*X**2*U[0, 3]**2 - 11202786*X**2*U[0, 3]*U[2, 1] + 7728048*X**2*U[0, 3]*U[2, 3] + 7348320*X**2*U[2, 1]**2 - 6797196*X**2*U[2, 1]*U[2, 3] + 2344464*X**2*U[2, 3]**2 + 121088016*X*Y*U[0, 1]*U[1, 0] + 31548468*X*Y*U[0, 1]*U[1, 2] - 44137044*X*Y*U[0, 1]*U[3, 0] - 11499537*X*Y*U[0, 1]*U[3, 2] - 44137044*X*Y*U[0, 3]*U[1, 0] - 23526552*X*Y*U[0, 3]*U[1, 2] + 16088121*X*Y*U[0, 3]*U[3, 0] + 8575518*X*Y*U[0, 3]*U[3, 2] + 31548468*X*Y*U[1, 0]*U[2, 1] - 11499537*X*Y*U[1, 0]*U[2, 3] + 8219689*X*Y*U[1, 2]*U[2, 1] - 6129646*X*Y*U[1, 2]*U[2, 3] - 23526552*X*Y*U[2, 1]*U[3, 0] - 6129646*X*Y*U[2, 1]*U[3, 2] + 8575518*X*Y*U[2, 3]*U[3, 0] + 4

In [13]:
F = FB + FC + F0
F

Matrix([
[B*X*Y*(6192*X**2*U[0, 1]**2 - 6966*X**2*U[0, 1]*U[0, 3] + 2592*X**2*U[0, 1]*U[2, 1] - 1458*X**2*U[0, 1]*U[2, 3] + 3096*X**2*U[0, 3]**2 - 1458*X**2*U[0, 3]*U[2, 1] + 1296*X**2*U[0, 3]*U[2, 3] + 1296*X**2*U[2, 1]**2 - 1458*X**2*U[2, 1]*U[2, 3] + 648*X**2*U[2, 3]**2 + 9409*X*Y*U[0, 1]*U[1, 0] + 3395*X*Y*U[0, 1]*U[1, 2] - 4171*X*Y*U[0, 1]*U[3, 0] - 1505*X*Y*U[0, 1]*U[3, 2] - 4171*X*Y*U[0, 3]*U[1, 0] - 3395*X*Y*U[0, 3]*U[1, 2] + 1849*X*Y*U[0, 3]*U[3, 0] + 1505*X*Y*U[0, 3]*U[3, 2] + 3395*X*Y*U[1, 0]*U[2, 1] - 1505*X*Y*U[1, 0]*U[2, 3] + 1225*X*Y*U[1, 2]*U[2, 1] - 1225*X*Y*U[1, 2]*U[2, 3] - 3395*X*Y*U[2, 1]*U[3, 0] - 1225*X*Y*U[2, 1]*U[3, 2] + 1505*X*Y*U[2, 3]*U[3, 0] + 1225*X*Y*U[2, 3]*U[3, 2] + 75078*X*U[0, 0]*U[0, 1] - 33282*X*U[0, 0]*U[0, 3] + 15714*X*U[0, 0]*U[2, 1] - 6966*X*U[0, 0]*U[2, 3] + 27090*X*U[0, 1]*U[0, 2] + 15714*X*U[0, 1]*U[2, 0] + 5670*X*U[0, 1]*U[2, 2] - 27090*X*U[0, 2]*U[0, 3] + 5670*X*U[0, 2]*U[2, 1] - 5670*X*U[0, 2]*U[2, 3] - 6966*X*U[0, 3]*U[2, 0] - 5670*X*U[0,

# Wypalanie nośników

Wypalanie nośników w ogólności opisane jest wzorem
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{g(\mathbf{r}, u(\mathbf{r}))}{\hbar \omega} \frac{P\,M(\mathbf{r})}{1-R} = \frac{\lambda}{h\,c} \, g(\mathbf{r},u(\mathbf{r})) \, \frac{P\,M(\mathbf{r})}{1-R},
$$
gdzie $P$ to moc emitowana, $R$ odbijalność zwierciadeł, $M(\mathbf{r})$ to unormowany bezwymiarowy profil modu, $g(\mathbf{r},u(\mathbf{r}))$ wzmocnienie.

PLaSK dostarcza bezpośrednio natężenie modu w obszarza czynnym $P(\mathbf{r})$. Zatem powyższe równanie ma postać
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{\lambda}{h\,c} \, g(\mathbf{r},u(\mathbf{r})) \, P(\mathbf{r}).
$$

Zakładamy, że na pojedynczym elemencie natężenie zmienia się liniowo. Z kolei wzmocnienie jest liniową funkcją koncentracji i na całym elemencie ma postać
$$
g(\mathbf{r},u(\mathbf{r})) \approx g(u_0) + g_u \left(u(\mathbf{r}) - u_0 \right) = g(u_0) - g_u\,u_0 + g_u\,u(\mathbf{r}).
$$
Daje to
$$
L_\mathrm{SHB}(\mathbf{r}) = \frac{\lambda}{h\,c} \, P(\mathbf{r}) \, g_u\,u(\mathbf{r}) - \frac{\lambda}{h\,c} \, P(\mathbf{r}) \, \left[ g_u\,u_0 - g(u_0) \right].
$$

In [14]:
P = sym.IndexedBase('P', shape=(2,2))
G = sym.IndexedBase('G', shape=(2,))
dG = sym.IndexedBase('dG', shape=(2,))
uG = sym.symbols('uG')


In [15]:
Pxy = [sym.simplify(sum(P[i, j] * La(i, x, zx) * La(j, y, zy) for i in range(len(zx)) for j in range(len(zy)))) for j in range(2)]

In [16]:
execfile('matrices2-shb.py')

In [17]:
KL

Matrix([
[      X*Y*(100*P[0, 0]*dG[0] + 100*P[0, 0]*dG[1] + 30*P[0, 1]*dG[0] + 30*P[0, 1]*dG[1] + 30*P[1, 0]*dG[0] + 30*P[1, 0]*dG[1] + 9*P[1, 1]*dG[0] + 9*P[1, 1]*dG[1])/1225,             X**2*Y*(150*P[0, 0]*dG[0] + 150*P[0, 0]*dG[1] + 45*P[0, 1]*dG[0] + 45*P[0, 1]*dG[1] + 70*P[1, 0]*dG[0] + 70*P[1, 0]*dG[1] + 21*P[1, 1]*dG[0] + 21*P[1, 1]*dG[1])/14700,          9*X*Y*(10*P[0, 0]*dG[0] + 10*P[0, 0]*dG[1] + 3*P[0, 1]*dG[0] + 3*P[0, 1]*dG[1] + 10*P[1, 0]*dG[0] + 10*P[1, 0]*dG[1] + 3*P[1, 1]*dG[0] + 3*P[1, 1]*dG[1])/4900,              -X**2*Y*(70*P[0, 0]*dG[0] + 70*P[0, 0]*dG[1] + 21*P[0, 1]*dG[0] + 21*P[0, 1]*dG[1] + 60*P[1, 0]*dG[0] + 60*P[1, 0]*dG[1] + 18*P[1, 1]*dG[0] + 18*P[1, 1]*dG[1])/14700,             X*Y**2*(150*P[0, 0]*dG[0] + 150*P[0, 0]*dG[1] + 70*P[0, 1]*dG[0] + 70*P[0, 1]*dG[1] + 45*P[1, 0]*dG[0] + 45*P[1, 0]*dG[1] + 21*P[1, 1]*dG[0] + 21*P[1, 1]*dG[1])/14700,                 3*X*Y**2*(15*P[0, 0]*dG[0] + 15*P[0, 0]*dG[1] + 7*P[0, 1]*dG[0] + 7*P[0, 1]*dG[1] + 15*P[1, 0]*dG

In [18]:
FL

Matrix([
[ X*Y*(49*uG*P[0, 0]*dG[0] + 49*uG*P[0, 0]*dG[1] + 21*uG*P[0, 1]*dG[0] + 21*uG*P[0, 1]*dG[1] + 21*uG*P[1, 0]*dG[0] + 21*uG*P[1, 0]*dG[1] + 9*uG*P[1, 1]*dG[0] + 9*uG*P[1, 1]*dG[1] - 49*G[0]*P[0, 0] - 21*G[0]*P[0, 1] - 21*G[0]*P[1, 0] - 9*G[0]*P[1, 1] - 49*G[1]*P[0, 0] - 21*G[1]*P[0, 1] - 21*G[1]*P[1, 0] - 9*G[1]*P[1, 1])/400],
[ X**2*Y*(21*uG*P[0, 0]*dG[0] + 21*uG*P[0, 0]*dG[1] + 9*uG*P[0, 1]*dG[0] + 9*uG*P[0, 1]*dG[1] + 14*uG*P[1, 0]*dG[0] + 14*uG*P[1, 0]*dG[1] + 6*uG*P[1, 1]*dG[0] + 6*uG*P[1, 1]*dG[1] - 21*G[0]*P[0, 0] - 9*G[0]*P[0, 1] - 14*G[0]*P[1, 0] - 6*G[0]*P[1, 1] - 21*G[1]*P[0, 0] - 9*G[1]*P[0, 1] - 14*G[1]*P[1, 0] - 6*G[1]*P[1, 1])/1200],
[ X*Y*(21*uG*P[0, 0]*dG[0] + 21*uG*P[0, 0]*dG[1] + 9*uG*P[0, 1]*dG[0] + 9*uG*P[0, 1]*dG[1] + 49*uG*P[1, 0]*dG[0] + 49*uG*P[1, 0]*dG[1] + 21*uG*P[1, 1]*dG[0] + 21*uG*P[1, 1]*dG[1] - 21*G[0]*P[0, 0] - 9*G[0]*P[0, 1] - 49*G[0]*P[1, 0] - 21*G[0]*P[1, 1] - 21*G[1]*P[0, 0] - 9*G[1]*P[0, 1] - 49*G[1]*P[1, 0] - 21*G[1]*P[1, 1])/400],
[X**2*Y